In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import graphviz

: 

In [ ]:
with open('input.txt') as f:
    data = f.read().splitlines()
data

: 

In [3]:
CFG = {}
for i in data:
    print(i)
    var = i[0]
    j = i.split(' | ')
    t = 1
    CFG[var] = []
    CFG[var].append(j[0].split(" → ")[1])
    if len(j) > 1:
        while (t < len(j)):
            CFG[var].append(j[t])
            t += 1
print(CFG)

S → AB
A → a
B → C | b
C → D
D → E
E → a
{'S': ['AB'], 'A': ['a'], 'B': ['C', 'b'], 'C': ['D'], 'D': ['E'], 'E': ['a']}


# Eliminate Unit Productions

In [4]:
def remove_unit(CFG):
    single_terminals = []
    for i in CFG:
        if len(CFG[i])==1 and CFG[i][0].islower():
            single_terminals.append(i)
    print(single_terminals)
    # Now, checking if any of the single terminals are in RHS of any other state
    while True:
        changed = False
        for i in CFG:
            for j in CFG[i]:
                for l in j:
                    if l in single_terminals:
                        k = single_terminals.index(l)
                        CFG[i].append(j.replace(single_terminals[k], CFG[single_terminals[k]][0]))
                        CFG[i].remove(j)
                        if len(CFG[i])==1:
                            single_terminals.append(i)
                        print(single_terminals)
                        changed = True
        if not changed:
            break
    print(CFG)
    return CFG

In [5]:
CFG = remove_unit(CFG)

['A', 'E']
['A', 'E', 'S']
['A', 'E', 'S', 'D']
['A', 'E', 'S', 'D', 'C']
['A', 'E', 'S', 'D', 'C']
{'S': ['aB'], 'A': ['a'], 'B': ['b', 'a'], 'C': ['a'], 'D': ['a'], 'E': ['a']}


# Elimate Useless Productions

In [6]:
def remove_useless(CFG):
    reachable_states = []
    given_states = []
    terminals = []
    for i in CFG:
        if i not in given_states:
            given_states.append(i)
        for j in CFG[i]:
            for l in j:
                if l not in reachable_states:
                    if l.isupper():
                        reachable_states.append(l)
                    else:
                        terminals.append(l)
    print(reachable_states)
    print(given_states)
    print(terminals)
    unreachable = []
    for i in given_states:
        if i=="S":
            continue
        elif i not in reachable_states:
            unreachable.append(i)
    print(unreachable)
    for i in unreachable:
        CFG.pop(i)
    print(CFG)
    return CFG

In [7]:
CFG = remove_useless(CFG)

['B']
['S', 'A', 'B', 'C', 'D', 'E']
['a', 'a', 'b', 'a', 'a', 'a', 'a']
['A', 'C', 'D', 'E']
{'S': ['aB'], 'B': ['b', 'a']}


# Eliminate Null Productions

In [8]:
def remove_null(CFG):
    # Step 1: Find nullable non-terminals
    nullable = set()
    for nonterm in CFG:
        if '' in CFG[nonterm]:
            nullable.add(nonterm)
    while True:
        new_nullable = set()
        for nonterm, prods in CFG.items():
            for prod in prods:
                if all(s in nullable for s in prod):
                    new_nullable.add(nonterm)
                    break
        if new_nullable == nullable:
            break
        nullable = new_nullable
    
    # Step 2: Construct new productions
    new_prods = {}
    for nonterm, prods in CFG.items():
        new_prods[nonterm] = set()
        for prod in prods:
            new_prod = ['']
            for s in prod:
                if s in nullable:
                    new_prod += [x + s for x in new_prod]
                else:
                    new_prod = [x + s for x in new_prod]
            new_prods[nonterm] |= set(new_prod)
    
    # Step 3: Remove epsilon productions and return the new CFG
    new_CFG = {}
    for nonterm, prods in new_prods.items():
        new_CFG[nonterm] = set()
        for prod in prods:
            if prod != '':
                new_CFG[nonterm].add(prod)
    return new_CFG


In [9]:
#CFG = {'S':['AB'], 'A':['aAA','bB',''], 'B':['bBB','']}
cfg = remove_null(CFG)
for nonterm, prods in cfg.items():
    print(nonterm, '->', ' | '.join(prods))


S -> aB
B -> b | a


In [17]:
with open('input.txt') as f:
    data = f.read().splitlines()
CFG = {}
for i in data:
    print(i)
    var = i[0]
    j = i.split(' | ')
    t = 1
    CFG[var] = []
    CFG[var].append(j[0].split(" → ")[1])
    if len(j) > 1:
        while (t < len(j)):
            CFG[var].append(j[t])
            t += 1
print(CFG)

S → AB
A → a
B → C | b
C → D
D → E
E → a
{'S': ['AB'], 'A': ['a'], 'B': ['C', 'b'], 'C': ['D'], 'D': ['E'], 'E': ['a']}


In [18]:
cfg1 = remove_unit(CFG)
cfg2 = remove_useless(cfg1)
cfg3 = remove_null(cfg2)
print("\n\n")
for nonterm, prods in cfg3.items():
    print(nonterm, '->', ' | '.join(prods))

['A', 'E']
['A', 'E', 'S']
['A', 'E', 'S', 'D']
['A', 'E', 'S', 'D', 'C']
['A', 'E', 'S', 'D', 'C']
{'S': ['aB'], 'A': ['a'], 'B': ['b', 'a'], 'C': ['a'], 'D': ['a'], 'E': ['a']}
['B']
['S', 'A', 'B', 'C', 'D', 'E']
['a', 'a', 'b', 'a', 'a', 'a', 'a']
['A', 'C', 'D', 'E']
{'S': ['aB'], 'B': ['b', 'a']}



S -> aB
B -> b | a


In [ ]:
def cfg_to_cnf(cfg):
    # Step 1: Add new start symbol if necessary
    if 'S' in cfg and any('S' in rhs for rhs in cfg.values()):
        new_start = 'S\''
        while new_start in cfg:
            new_start += '\''
        cfg[new_start] = ['S']
        del cfg['S']
        for symbol, rhs in cfg.items():
            cfg[symbol] = [new_start if s == 'S' else s for s in rhs]
    
    # Step 2: Remove null productions (not needed)
    
    # Step 3: Remove unit productions (not needed)
    
    # Step 4: Replace A → aB with A → XB, X → a
    new_rules = {}
    for symbol, rhs in cfg.items():
        new_rhs = []
        for prod in rhs:
            if len(prod) == 1:
                new_rhs.append(prod)
            else:
                for i, s in enumerate(prod):
                    if s.islower():
                        new_nonterm = f'{symbol}_{i}'
                        if new_nonterm not in new_rules:
                            new_rules[new_nonterm] = [s]
                        new_rhs.append(new_nonterm)
                        break
                    elif i == len(prod) - 1:
                        new_rhs.append(prod)
        cfg[symbol] = new_rhs
    for symbol, rhs in new_rules.items():
        cfg[symbol] = rhs
    
    # Step 3: Replace A → B1B2...Bn with A → B1C, C → B2...Bn
    new_rules = {}
    for symbol, rhs in cfg.items():
        new_rhs = []
        for prod in rhs:
            if len(prod) <= 2:
                new_rhs.append(prod)
            else:
                new_nonterm = f'{symbol}_1'
                if new_nonterm not in new_rules:
                    new_rules[new_nonterm] = prod[1:]
                new_rhs.append(prod[:2] + [new_nonterm])
                for i in range(2, len(prod)):
                    prev_nonterm, new_nonterm = new_nonterm, f'{symbol}_{i}'
                    if new_nonterm not in new_rules:
                        new_rules[new_nonterm] = prod[i:]
                    new_rhs.append([prev_nonterm, new_nonterm])
        cfg[symbol] = new_rhs
    for symbol, rhs in new_rules.items():
        cfg[symbol] = [rhs]
    
    return cfg
CFG = {'S': ['AB'], 'A': ['a'], 'B': ['C', 'b'], 'C': ['D'], 'D': ['E'], 'E': ['a']}
cfg_to_cnf(CFG)

: 

# Plot the finite Automata

In [19]:
import graphviz

def plot_automaton(grammar):
    """Plot the finite automaton corresponding to a CFG."""
    # Create a new graph
    graph = graphviz.Digraph(format='png')

    # Add states and transitions for each production in the CFG
    for var, prods in grammar.items():
        # Add the state for the variable
        graph.node(var)
        for prod in prods:
            # If the production is a single terminal symbol, add a transition to a new dummy state
            if len(prod) == 1 and prod[0].islower():
                graph.edge(var, 'D0', label=prod[0])
            else:
                # Add a transition for each symbol in the production
                for i in range(len(prod)):
                    graph.edge(var, prod[i])
                    # If the symbol is a nonterminal that produces a single terminal symbol, add a transition to a dummy state
                    if len(prod[i]) == 1 and prod[i].islower():
                        graph.edge(prod[i], 'D0', label=prod[i])

    # Add a dummy state for single-terminal productions
    graph.node('D0', shape='diamond')

    # Set the graph attributes
    graph.attr(rankdir='LR', size='8,5')

    # Render and display the graph
    graph.render(directory="C:\\Users\\DEVANSH\\Downloads\\THOC_Innovative", view=True)  
    print(graph)

In [20]:
plot_automaton(cfg)

digraph {
	S
	S -> a
	a -> D0 [label=a]
	S -> B
	B
	B -> D0 [label=b]
	B -> D0 [label=a]
	D0 [shape=diamond]
	rankdir=LR size="8,5"
}

